<a href="https://colab.research.google.com/github/kindustrii/doiit-text-klassifizierung-rnn/blob/main/doiit_text_klassifizierung_mit_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Unser Neuronales Netz wir Filmbewertungen, die als Text vorliegen, darauf untersuchen ob es sich um eine positive oder um eine negative Bewertung handelt.

Als Trainingsdaten für das Neuronale Netz werden wird den IMDB-Datensatz verwenden. Es gibt mehrere Wege diesen Datensatz zu laden. Wir verwenden eine Variante mittleren Aufwandes und nutzen dafür das Package tensorflow-datasets. Da dieses Package nicht standardmäßig installiert ist, führen wir zunächst die Installation aus. Danach importierem notwendigen Bibliotheken.

In [1]:
!pip install -q tensorflow-datasets

In [2]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

# IMDB-Datensatz laden

Der IMDB-Datensatz besteht aus 50.000 Filmbewertung (in englischer Sprache) mit je 25.000 positiven und negativen. Eine negative Bewertung ist mit einer 0 markiert und eine positive Bewertung ist mit einer 1 markiert.

In [3]:
# laden des gesamten Datensatzes
dataset, info = tfds.load('imdb_reviews',with_info=True,
                          as_supervised=True)

# Aufteilen des gesamten Datensatzen in Trainings- und Testdaten
# Die Testdaten werden zur Seite gelegt und wir verweden
# diese zum Schluss um das Neuronale Netz zu testen
# Wir teilen in diesem Fall den Datensatz in dem Verhältnis 50/50 auf
train_data, test_data = dataset['train'],dataset['test'] 


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteR3EOV6/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteR3EOV6/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteR3EOV6/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


Sehen wir uns uns mal die ersten 5 Filmbewertung aus den Trainingsdaten an.

In [4]:
for bewertung,markierung in train_data.take(5):
  print('Bewertung:  {}'.format(bewertung.numpy()))
  print('Markierung: {}\n'.format(markierung.numpy()))

Bewertung:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Markierung: 0

Bewertung:  b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the fil

Das sieht doch ganz gut aus. Für die ersten fünf Filmbewertungen passt die Markierung 0 (negativ) und 1 (positiv) zu dem jeweiligen Text.

Diese Markierungen wurden von Menschen erstellt. D.h. ein Mensch hat sich die Bewertungen durchgelesen und je nachdem ob darin der Film gut oder schlecht bewertet wurde, mit einer 1 oder mit einer 0 markiert. 

Unsere Aufgabe besteht nun darin ein Neuronales Netz zu erstellen, das in der gleichen Weise wie der Mensch eine Filmbewertung bekommt und diese entweder mit einer 0 oder mit einer 1 markiert. Damit es diese Aufgabe erfolgreich durchführen kann, werden wir in das Neuronalen Netz die Trainingsdaten als Pärchen (Bewertung/Markierung) einspeisen. In dem Training wird, vereinfacht gesagt, das NN die Muster von positiven und negativen Bewertungen lernen und später eine unbekannte Filmbewertung auf diese Muster durchsuchen und so die Entscheidung treffen, ob mit einer 1 oder einer 0 Markiert werden soll.

In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [6]:
train_data = train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
for bewertung, markierung in train_data.take(1):
  print('Bewertung: {}\n'.format(bewertung.numpy()[:3]))
  print('Markierung: {}\n'.format(markierung.numpy()[:3]))

Bewertung: [b"Cross-eyed is a very original and funny movie. I think Adam Jones brings a refreshing new set of eyes to the comedy genre and really reinvents it in a good way. This film is smart, concise, and consistently entertaining and funny. As a writer/director, Jones exhibits complete control over his characters who are both absurd and lovable. The story is definitely something you haven't seen before which is good. It's unique and fun, and manages to work in visually fantastic elements as well as the long lost slapstick genre together to form a hearty comedy.<br /><br />A very promising first film."
 b'I feel it is my duty as a lover of horror films to warm other people about this horrible and very very bad "horror" film. Don\'t waste your time or money on this film, the acting is bad, the story is just one of the worst i have come across and the script was just awful. Nothing about it was good, you end up thinking to yourself why am i watching this crap. The plot had so many hol

# Den Text codieren

Die Daten (Filmbewertungen), die vom Neuronalen Netz verarbeitet werden müssen stehen uns in Form von Text zur Verfügung. Bevor diese ans NN übergeben werden können, muss der Text vorverarbeitet und in eine bestimmte Form gebracht werden. Wir verwenden hierfür die sogenannte Wort Vektorisierung, die wir als Verarbeitungsschicht vor die Input-Schicht des Neuronalen Netzes setzen.

Mit dieser Technik erstellen wir zunächst einmal eine Vokabelliste. Dafür werden alle 50.000 Filmbewertungen durchlaufen und jedes einzelne Wort das vorkommt, einmalig in die Liste aufgenommen. Jedes Wort, dass nun in der Vokabelliste steht, erhält dann eine eigene Zahl zugewiesen, sodass jeder Satz mithilfe dieser List in eine Zahlenfolge übersetzt werden kann.

Hier ein Beispiel zur Verdeutlichung.

Unser **Satz** lautet: kindustrii ist super, super cool. 

Daraus entsteht die **Vokabelliste**: [kindustrii, ist, super, cool]. 

Jedem Wort in der Vokabelliste wird **eine Zahl zugeordnet**: {'kindustrii': 0, 'ist': 1, 'super': 2, 'cool': 3}

Übersetzer Satz, bzw. **Vektor**: [0, 1, 2, 2, 3]



In [8]:
VOCAB_SIZE = 1000
# Der Encoder durchläuft alle Filmbewertungen und sammelt alle Wörter ein
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE
)
encoder.adapt(train_data.map(lambda text, label: text))

In [9]:
# Wir sehen uns 50 Wörter der Vokabelliste an
vocab = np.array(encoder.get_vocabulary())
vocab[500:550]

array(['starts', 'son', 'kill', 'game', 'act', 'sometimes', 'side',
       'viewer', 'town', 'horrible', 'parts', 'car', 'actress', 'soon',
       'child', 'ones', 'eyes', 'expect', 'obviously', 'flick',
       'themselves', 'directed', 'thinking', 'heart', 'art', 'brilliant',
       'stories', 'ill', 'decent', 'highly', 'run', 'feeling', 'myself',
       'genre', 'late', 'blood', 'stuff', 'fight', 'says', 'close',
       'took', 'city', 'except', 'cannot', 'heard', 'hand', 'leave',
       'killed', 'kid', 'matter'], dtype='<U14')

In [10]:
# Hier sehen wir uns mal drei Filmbewertungen an, die als Zahlen-Vektor dargestellt sind
# Hinweis: Alle Filmbewertungen werden auf dieselbe Länge gebracht, in dem kurze
# Texte hinten mit 0en aufgefüllt werden

encoded_example = encoder(bewertung)[:3].numpy()
encoded_example

array([[  1,   7,   4, ...,   0,   0,   0],
       [ 10, 232,   9, ...,   0,   0,   0],
       [ 10, 237,   1, ...,   0,   0,   0]])

Jede der drei oberen Zeilen repräsentiert eine Filmbewertung. An den hinteren Nullen erkennt man, dass die Bewertungen weniger Wörter enthalten als die längste aller Bewertungen und deshalb aufgefüllt werden. Diese Nullen werden später im Inneren des Neuronalen Netzes herausgefiltert, um das Ergebnis nicht zu beeinflussen.

Vergewissern wir uns noch kurz, dass die drei oberen Vektoren tatsächlich gleich viele Elemente enthalten.

In [22]:
for i in range(3):
  print('Laenge der Filmbewertung {}: {}'.format(i,len(encoded_example[i])))

Laenge der Filmbewertung 0: 860
Laenge der Filmbewertung 1: 860
Laenge der Filmbewertung 2: 860


# Das Neuronale Netz erstellen

In der Code-Zeile [11] wird das gesamte Neuronale Netz erstellt, dass insgesamt aus 5 Schichten besteht.  

In [11]:
model = tf.keras.Sequential()
model.add(encoder)
model.add(tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()),
                                    output_dim=64,
                                    mask_zero=True))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [13]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.00387018]


In [14]:
len(encoder.get_vocabulary())

1000